In [3]:
import pandas as pd 
import numpy as np

In [22]:
# here we read csv file 
df=pd.read_csv('covid_19_data.csv').dropna()
# now just remove the space to easy acess the column name
df.columns=df.columns.str.strip().str.lower().str.replace('/','_').str.replace(' ','_')
# set this to columnb to date time
df.observationdate  = pd.to_datetime(df.observationdate)  
df.last_update = pd.to_datetime(df.last_update ,format='mixed')
df.head(2)


,sno,observationdate,province_state,country_region,last_update,confirmed,deaths,recovered
0,1,2020-01-22,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0


# top 10 country with max deaths

In [14]:


df.groupby('country_region').deaths.sum()\
.reset_index(name='count').nlargest(10,'count')

,country_region,count
27,US,123303762.0
2,Brazil,72339029.0
11,India,44256013.0
17,Mexico,42918161.0
26,UK,27316263.0
13,Italy,24852224.0
22,Russia,18271229.0
23,Spain,18124340.0
6,Colombia,13962927.0
9,Germany,13150115.0


# top 3 city in each country with max death

In [43]:


df.groupby(['country_region','province_state']).apply(lambda x: x.nlargest(3,'deaths')).reset_index(drop=True)

,sno,observationdate,province_state,country_region,last_update,confirmed,deaths,recovered
0,15550,2020-04-15,Australian Capital Territory,Australia,2020-04-15 23:04:26,103.0,3.0,72.0
1,15874,2020-04-16,Australian Capital Territory,Australia,2020-04-16 23:38:19,103.0,3.0,82.0
2,16198,2020-04-17,Australian Capital Territory,Australia,2020-04-17 23:38:48,103.0,3.0,81.0
3,4802,2020-03-10,Diamond Princess cruise ship,Australia,2020-02-29 02:03:10,0.0,0.0,0.0
4,5023,2020-03-11,Diamond Princess cruise ship,Australia,2020-02-29 02:03:10,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2193,305660,2021-05-28,Zaporizhia Oblast,Ukraine,2021-05-29 04:20:41,102471.0,2323.0,94730.0
2194,304895,2021-05-27,Zaporizhia Oblast,Ukraine,2021-05-28 04:20:36,102276.0,2313.0,94167.0
2195,306428,2021-05-29,Zhytomyr Oblast,Ukraine,2021-05-30 04:20:55,87550.0,1738.0,83790.0
2196,305663,2021-05-28,Zhytomyr Oblast,Ukraine,2021-05-29 04:20:41,87473.0,1734.0,83575.0


# use of group by
# in which month country have max death

In [23]:
df.groupby(['country_region',df.last_update.dt.strftime('%Y-%b')])\
.apply(lambda x: x.deaths.sum()).reset_index(name='p').sort_values('p',ascending=False)


,country_region,last_update,p
98,US,2021-Apr,104413544.0
9,Brazil,2021-Apr,59358479.0
66,Mexico,2021-Apr,36317391.0
45,India,2021-Apr,36194211.0
91,UK,2021-Apr,23476671.0
...,...,...,...
27,Denmark,2020-May,0.0
54,Macau,2020-Jan,0.0
55,Macau,2020-Mar,0.0
56,Macau,2020-May,0.0


# in each country top 2 month with max death

In [40]:
df.groupby(['country_region',df.last_update.dt.to_period('M')])\
.deaths.sum().reset_index(name='no death').groupby('country_region')\
.apply(lambda x: x.nlargest(2,'no death'))[['last_update','no death']]\
.reset_index()[['country_region','last_update','no death']].sort_values('country_region').head(10)

,country_region,last_update,no death
0,Australia,2021-04,238651.0
1,Australia,2021-05,27300.0
2,Belgium,2021-04,3480030.0
3,Belgium,2021-05,739245.0
4,Brazil,2021-04,59358479.0
5,Brazil,2021-05,12980550.0
6,Canada,2021-04,4869164.0
7,Canada,2021-05,745304.0
8,Chile,2021-04,4877403.0
9,Chile,2021-05,830261.0


# use of table Join
# show date on which death are more than avg death of that month

In [39]:
df2 = df.groupby(['country_region',df.last_update.dt.strftime('%Y-%b')])\
.deaths.mean().reset_index(name='avg_of_month')[['country_region','avg_of_month']]

df3=pd.merge(df,df2,on='country_region')
df3[(df3.deaths>df3.avg_of_month)&(df3.avg_of_month>3)].sort_values(by=['country_region','last_update'])\
[['country_region','last_update','province_state','deaths','confirmed','avg_of_month']]

,country_region,last_update,province_state,deaths,confirmed,avg_of_month
355947,Australia,2020-03-08 05:31:00,New South Wales,8.0,1617.0,7.431034
356003,Australia,2020-03-08 05:31:00,New South Wales,8.0,1791.0,7.431034
356059,Australia,2020-03-08 05:31:00,New South Wales,8.0,2032.0,7.431034
356115,Australia,2020-03-31 23:49:27,New South Wales,8.0,2032.0,7.431034
356171,Australia,2020-04-01 22:04:58,New South Wales,9.0,2182.0,7.431034
...,...,...,...,...,...,...
737024,Ukraine,2021-05-30 04:20:55,Volyn Oblast,1148.0,61187.0,525.743521
737026,Ukraine,2021-05-30 04:20:55,Zakarpattia Oblast,1586.0,61611.0,525.743521
737028,Ukraine,2021-05-30 04:20:55,Zaporizhia Oblast,2335.0,102641.0,525.743521
737029,Ukraine,2021-05-30 04:20:55,Zaporizhia Oblast,2335.0,102641.0,1830.298765


# for china  top 2 day with max death

In [26]:

df[df.country_region == 'Mainland China'].nlargest(2,'deaths')\
[['country_region','last_update','province_state','deaths']].reset_index(drop=True)

,country_region,last_update,province_state,deaths
0,Mainland China,2020-04-17 23:38:48,Hubei,4512.0
1,Mainland China,2020-04-18 22:40:18,Hubei,4512.0


# show data of that  day where confirmd case is more than death and death are above 10 and confirmed case also above 10

In [19]:
df.loc[(df.deaths<df.confirmed)&(df.deaths>10)&(df.confirmed>10)]

,sno,observationdate,province_state,country_region,last_update,confirmed,deaths,recovered
13,14,2020-01-22,Hubei,Mainland China,2020-01-22 17:00:00,444.0,17.0,28.0
49,50,2020-01-23,Hubei,Mainland China,2020-01-23 17:00:00,444.0,17.0,28.0
53,54,2020-01-23,Hubei,Mainland China,2020-01-23 17:00:00,444.0,17.0,28.0
88,89,2020-01-24,Hubei,Mainland China,2020-01-24 17:00:00,549.0,24.0,31.0
131,132,2020-01-25,Hubei,Mainland China,2020-01-25 17:00:00,761.0,40.0,32.0
...,...,...,...,...,...,...,...,...
306423,306424,2021-05-29,Zakarpattia Oblast,Ukraine,2021-05-30 04:20:55,61611.0,1586.0,58882.0
306424,306425,2021-05-29,Zaporizhia Oblast,Ukraine,2021-05-30 04:20:55,102641.0,2335.0,95289.0
306425,306426,2021-05-29,Zeeland,Netherlands,2021-05-30 04:20:55,29147.0,245.0,0.0
306427,306428,2021-05-29,Zhytomyr Oblast,Ukraine,2021-05-30 04:20:55,87550.0,1738.0,83790.0


# use of filter 
# details of that country , day etc   in which monthly confirmed case are more then 1000000 

In [25]:
df.groupby([df.last_update.dt.month,'country_region']).filter(lambda x: x.confirmed.sum()>1000000)


,sno,observationdate,province_state,country_region,last_update,confirmed,deaths,recovered
513,514,2020-02-01,Hubei,Mainland China,2020-02-01 11:53:00,7153.0,249.0,168.0
514,515,2020-02-01,Zhejiang,Mainland China,2020-02-01 10:53:00,599.0,0.0,21.0
515,516,2020-02-01,Guangdong,Mainland China,2020-02-01 14:23:00,535.0,0.0,14.0
516,517,2020-02-01,Henan,Mainland China,2020-02-01 01:52:00,422.0,2.0,3.0
517,518,2020-02-01,Hunan,Mainland China,2020-02-01 11:03:00,389.0,0.0,8.0
...,...,...,...,...,...,...,...,...
306424,306425,2021-05-29,Zaporizhia Oblast,Ukraine,2021-05-30 04:20:55,102641.0,2335.0,95289.0
306425,306426,2021-05-29,Zeeland,Netherlands,2021-05-30 04:20:55,29147.0,245.0,0.0
306426,306427,2021-05-29,Zhejiang,Mainland China,2021-05-30 04:20:55,1364.0,1.0,1324.0
306427,306428,2021-05-29,Zhytomyr Oblast,Ukraine,2021-05-30 04:20:55,87550.0,1738.0,83790.0


# use of apply function 
# for each  month which country have max number of death for that month

In [21]:
df.groupby([df.last_update.dt.to_period('M'),'country_region']).deaths.sum().reset_index()\
.groupby('last_update').apply(lambda x: x.nlargest(2,'deaths'))\
.iloc[:,1:].reset_index(level=1,drop=True)

,country_region,deaths
last_update,,
2020-01,Mainland China,905.0
2020-01,Australia,0.0
2020-02,Mainland China,46382.0
2020-02,Others,41.0
2020-03,Mainland China,98372.0
2020-03,US,18447.0
2020-04,US,908408.0
2020-04,Mainland China,113537.0
2020-05,US,405057.0


# use of string concat , round , data type change 
# for each month percentage change of death 

In [24]:
df.groupby(df.last_update.dt.to_period('M'))['deaths']\
.sum().reset_index().assign(percentage_change= lambda x: np.where(x.deaths.shift(1).isnull(),0, 
(((x.deaths-x.deaths.shift(1))/x.deaths)*100).round(2).astype(str)+'%') )

,last_update,deaths,percentage_change
0,2020-01,905.0,0
1,2020-02,46475.0,98.05%
2,2020-03,122320.0,62.01%
3,2020-04,1075558.0,88.63%
4,2020-05,461948.0,-132.83%
5,2021-04,374753970.0,99.88%
6,2021-05,71371338.0,-425.08%


,country_region,last_update,province_state,deaths,confirmed,avg_of_month
355947,Australia,2020-03-08 05:31:00,New South Wales,8.0,1617.0,7.431034
356003,Australia,2020-03-08 05:31:00,New South Wales,8.0,1791.0,7.431034
356059,Australia,2020-03-08 05:31:00,New South Wales,8.0,2032.0,7.431034
356115,Australia,2020-03-31 23:49:27,New South Wales,8.0,2032.0,7.431034
356171,Australia,2020-04-01 22:04:58,New South Wales,9.0,2182.0,7.431034
...,...,...,...,...,...,...
737024,Ukraine,2021-05-30 04:20:55,Volyn Oblast,1148.0,61187.0,525.743521
737026,Ukraine,2021-05-30 04:20:55,Zakarpattia Oblast,1586.0,61611.0,525.743521
737028,Ukraine,2021-05-30 04:20:55,Zaporizhia Oblast,2335.0,102641.0,525.743521
737029,Ukraine,2021-05-30 04:20:55,Zaporizhia Oblast,2335.0,102641.0,1830.298765
